<a href="https://colab.research.google.com/github/pushkar243/AWS_Config/blob/master/GNN_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torch-geometric ogb scikit-learn


In [2]:
#Import required libraries
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
import numpy as np
from ogb.linkproppred import LinkPropPredDataset, Evaluator


In [3]:
##Define GCN Model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [4]:
#Step 4: Load the Dataset
dataset = LinkPropPredDataset(name="ogbl-collab")
split_edge = dataset.get_edge_split()
data = dataset[0]


/usr/local/lib/python3.10/dist-packages/ogb/linkproppred/dataset.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.graph = torch.load(pre_processed_file_path, 'rb')


In [5]:
#Step 5: Move Data to the Device
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert numpy arrays to torch tensors and move them to the device
edge_index = torch.tensor(data['edge_index'], dtype=torch.long).to(device)
node_feat = torch.tensor(data['node_feat'], dtype=torch.float).to(device)  # Node features



In [6]:
#Step6: Initialize the model and optimizer
model = GCN(in_channels=node_feat.size(1), hidden_channels=64, out_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [7]:
#Generate Negative Samples
from torch_geometric.utils import negative_sampling

# Sample negative edges
num_nodes = node_feat.size(0)
neg_edge_index = negative_sampling(edge_index, num_nodes=num_nodes)


In [8]:
#Create Labels
import torch

# Create labels for positive and negative samples
num_pos_edges = edge_index.size(1)
num_neg_edges = neg_edge_index.size(1)

# True edges labeled as 1
pos_labels = torch.ones(num_pos_edges)

# False edges labeled as 0
neg_labels = torch.zeros(num_neg_edges)

# Concatenate both positive and negative labels
labels = torch.cat([pos_labels, neg_labels], dim=0).to(device)


In [9]:
#compute score and loss
# Forward pass
out = model(node_feat, edge_index)

# Get node embeddings for positive and negative edges
pos_out = out[edge_index[0]]  # Source node embeddings for positive edges
pos_out = torch.sum(pos_out * out[edge_index[1]], dim=1)  # Dot product for edge scores

neg_out = out[neg_edge_index[0]]  # Source node embeddings for negative edges
neg_out = torch.sum(neg_out * out[neg_edge_index[1]], dim=1)  # Dot product for edge scores

# Concatenate positive and negative edge scores
edge_scores = torch.cat([pos_out, neg_out], dim=0)

# Compute loss
criterion = torch.nn.BCEWithLogitsLoss()
loss = criterion(edge_scores, labels)


In [10]:
#Complete Training Loop
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch.optim.lr_scheduler import ReduceLROnPlateau # Import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Prepare data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
edge_index = torch.tensor(data['edge_index'], dtype=torch.long).to(device)
node_feat = torch.tensor(data['node_feat'], dtype=torch.float).to(device)

# Generate negative samples
num_nodes = node_feat.size(0)
neg_edge_index = negative_sampling(edge_index, num_nodes=num_nodes)

# Create labels
num_pos_edges = edge_index.size(1)
num_neg_edges = neg_edge_index.size(1)
pos_labels = torch.ones(num_pos_edges).to(device)
neg_labels = torch.zeros(num_neg_edges).to(device)
labels = torch.cat([pos_labels, neg_labels], dim=0).to(device)

# Initialize model, optimizer, and loss function
model = GCN(in_channels=node_feat.size(1), hidden_channels=64, out_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)
criterion = torch.nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    out = model(node_feat, edge_index)

    # Get node embeddings for positive and negative edges
    pos_out = out[edge_index[0]]
    pos_out = torch.sum(pos_out * out[edge_index[1]], dim=1)

    neg_out = out[neg_edge_index[0]]
    neg_out = torch.sum(neg_out * out[neg_edge_index[1]], dim=1)

    # Concatenate positive and negative edge scores
    edge_scores = torch.cat([pos_out, neg_out], dim=0)

    # Compute loss
    loss = criterion(edge_scores, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs
    if epoch % 2 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluation (optional)
model.eval()
with torch.no_grad():
    out = model(node_feat, edge_index)

    # Get embeddings for positive and negative edges
    pos_out = out[edge_index[0]]
    pos_out = torch.sum(pos_out * out[edge_index[1]], dim=1)

    neg_out = out[neg_edge_index[0]]
    neg_out = torch.sum(neg_out * out[neg_edge_index[1]], dim=1)

    # Compute metrics
    edge_scores = torch.cat([pos_out, neg_out], dim=0)
    predictions = torch.sigmoid(edge_scores).cpu().numpy()
    auc = roc_auc_score(labels.cpu().numpy(), predictions)
    print(f'AUC Score: {auc}')


Epoch 0, Loss: 0.8311553001403809
Epoch 2, Loss: 0.6295490860939026
Epoch 4, Loss: 0.613135814666748
Epoch 6, Loss: 0.5979081392288208
Epoch 8, Loss: 0.5793540477752686
Epoch 10, Loss: 0.5637637376785278
Epoch 12, Loss: 0.5512006282806396
Epoch 14, Loss: 0.5390623807907104
Epoch 16, Loss: 0.5284470915794373
Epoch 18, Loss: 0.5163910388946533
Epoch 20, Loss: 0.5038124322891235
Epoch 22, Loss: 0.49486276507377625
Epoch 24, Loss: 0.49004825949668884
Epoch 26, Loss: 0.48683062195777893
Epoch 28, Loss: 0.48093435168266296
Epoch 30, Loss: 0.4747813940048218
Epoch 32, Loss: 0.4690723419189453
Epoch 34, Loss: 0.4655914902687073
Epoch 36, Loss: 0.4650138020515442
Epoch 38, Loss: 0.4641154110431671
Epoch 40, Loss: 0.46317434310913086
Epoch 42, Loss: 0.461264431476593
Epoch 44, Loss: 0.45880815386772156
Epoch 46, Loss: 0.45658212900161743
Epoch 48, Loss: 0.4545833170413971
Epoch 50, Loss: 0.45324039459228516
Epoch 52, Loss: 0.45210692286491394
Epoch 54, Loss: 0.4508911073207855
Epoch 56, Loss: 0.

In [11]:
# Ensure that 'device' is set correctly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

val_edges = torch.tensor(split_edge['valid']['edge'], dtype=torch.long).to(device)
val_neg_edges = torch.tensor(split_edge['valid']['edge_neg'], dtype=torch.long).to(device)
# Assuming split_edge is already defined
val_edges = torch.tensor(split_edge['valid']['edge'], dtype=torch.long).to(device)
val_neg_edges = torch.tensor(split_edge['valid']['edge_neg'], dtype=torch.long).to(device)

# Forward pass with validation edges
model.eval()
with torch.no_grad():
    out = model(node_feat, edge_index)

    # Get embeddings for positive and negative validation edges
    pos_val_out = out[val_edges[:, 0]] * out[val_edges[:, 1]]
    pos_val_out = torch.sum(pos_val_out, dim=1)

    neg_val_out = out[val_neg_edges[:, 0]] * out[val_neg_edges[:, 1]]
    neg_val_out = torch.sum(neg_val_out, dim=1)

    # Combine and calculate the AUC
    val_scores = torch.cat([pos_val_out, neg_val_out], dim=0)
    val_labels = torch.cat([torch.ones(pos_val_out.size(0)), torch.zeros(neg_val_out.size(0))], dim=0).to(device)

    val_predictions = torch.sigmoid(val_scores).cpu().numpy()
    auc = roc_auc_score(val_labels.cpu().numpy(), val_predictions)
    print(f'Validation AUC: {auc}')


Validation AUC: 0.9437779717562079
